In [13]:
import numpy as np 
import pandas as pd

#### Functions

1. User-preferences Weights Calculations - Fuzzy Logic
2. Service QoS Weights Calculations - AHP
3. Weights Normalisation such ∑ w_i = 1
4. Final Weights Calculation RAT layer
5. MEW
6. SAW

#### 1. Fuzzy Membership Funtion

In [14]:
def fuzzy_membership(criteria_value, preference, min_value, max_value):
    """
    Calculate the fuzzy membership value for a given criteria.
    
    :param criteria_value: The value of the criteria to evaluate
    :param preference: 'low', 'medium', or 'high'
    :param min_value: The minimum value for the criteria range
    :param max_value: The maximum value for the criteria range
    :return: Membership value between 0 and 1
    """
    
    def low_mf(x, a, b):
        return np.where(x <= a, 1, max(0, (b - x) / (b - a)))
    
    def medium_mf(x, a, b, c):
        return max(0, min((x - a) / (b - a), (c - x) / (c - b)))
    
    def high_mf(x, b, c):
        return np.where(x >= c, 1, max(0, (x - b) / (c - b)))
    
    # Calculate the range
    value_range = max_value - min_value
    
    # Set ratios for a, b, c based on the range
    if preference == 'low':
        a = min_value + 0.2 * value_range
        b = min_value + 0.4 * value_range
        return low_mf(criteria_value, a, b)
    
    elif preference == 'medium':
        a = min_value + 0.3 * value_range
        b = min_value + 0.5 * value_range
        c = min_value + 0.7 * value_range
        return medium_mf(criteria_value, a, b, c)
    
    elif preference == 'high':
        b = min_value + 0.6 * value_range
        c = min_value + 0.8 * value_range
        return high_mf(criteria_value, b, c)
    
    else:
        raise ValueError("Preference must be 'low', 'medium', or 'high'")

#### 2. AHP Function

In [15]:
def ahp_weights_and_consistency(matrix):
    """
    Calculate weights and Consistency Index for AHP pairwise comparison matrix.
    
    Args:
    matrix (np.array): Square pairwise comparison matrix
    
    Returns:
    tuple: (weights, consistency_index)
    """
    # Check if the matrix is square
    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Matrix must be square")
    
    n = matrix.shape[0]
    
    # Calculate the principal eigenvalue and eigenvector
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_index = np.argmax(eigenvalues)
    max_eigenvalue = eigenvalues[max_index].real
    eigenvector = eigenvectors[:, max_index].real
    
    # Normalize the eigenvector to get the weights
    weights = eigenvector / np.sum(eigenvector)
    
    # Calculate the Consistency Index
    ci = (max_eigenvalue - n) / (n - 1)
    
    # Calculate the Random Index
    ri_values = {1: 0, 2: 0, 3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}
    ri = ri_values.get(n, 1.49)  # Use 1.49 for n > 10
    
    # Calculate the Consistency Ratio
    cr = ci / ri if ri != 0 else 0
    
    return weights, ci, cr

#### 3. Weights Normalisation

#### 4. Final Weights Calculation

#### 5. MEW

In [246]:
def ahp_weights_and_consistency(matrix):
    """
    Calculate weights and Consistency Index for AHP pairwise comparison matrix.
    
    Args:
    matrix (np.array): Square pairwise comparison matrix
    
    Returns:
    tuple: (weights, consistency_index)
    """
    # Check if the matrix is square
    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Matrix must be square")
    
    n = matrix.shape[0]
    
    # Calculate the principal eigenvalue and eigenvector
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_index = np.argmax(eigenvalues)
    max_eigenvalue = eigenvalues[max_index].real
    eigenvector = eigenvectors[:, max_index].real
    
    # Normalize the eigenvector to get the weights
    weights = eigenvector / np.sum(eigenvector)
    
    # Calculate the Consistency Index
    ci = (max_eigenvalue - n) / (n - 1)
    
    # Calculate the Random Index
    ri_values = {1: 0, 2: 0, 3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}
    ri = ri_values.get(n, 1.49)  # Use 1.49 for n > 10
    
    # Calculate the Consistency Ratio
    cr = ci / ri if ri != 0 else 0
    
    return weights, ci, cr

#### 6. SAW

In [17]:
def calculate_saw_scores(decision_matrix, weights):
    """
    Calculate scores using the Simple Additive Weighting (SAW) method.
    
    Parameters:
    decision_matrix (numpy.ndarray): A normalized decision matrix where rows represent alternatives and columns represent criteria.
    weights (numpy.ndarray): An array of weights for each criterion.
    
    Returns:
    numpy.ndarray: An array of SAW scores for each alternative.
    """
    
    # Ensure inputs are numpy arrays
    decision_matrix = np.array(decision_matrix)
    weights = np.array(weights)
    
    # Check if the number of weights matches the number of criteria
    if decision_matrix.shape[1] != weights.shape[0]:
        raise ValueError("The number of weights must match the number of criteria (columns) in the decision matrix.")
    
    # Calculate SAW scores
    saw_scores = np.sum(decision_matrix * weights, axis=1)
    
    return saw_scores

#### DATA

In [ ]:
#import pandas as pd
#import numpy as np
#from ahp_weights_and_consistency import ahp_weights_and_consistency  # Import the function we created earlier

def read_and_process_matrices(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path, header=None)
    
    # Initialize variables
    matrices = {}
    current_service = None
    current_matrix = []
    
    # Process the dataframe
    for index, row in df.iterrows():
        if pd.notna(row[0]) and pd.isna(row[1]):  # New service
            if current_service and current_matrix:
                matrices[current_service] = np.array(current_matrix)
            current_service = row[0]
            current_matrix = []
        elif pd.notna(row[0]) and pd.notna(row[1]):  # Matrix row
            current_matrix.append(row[1:5].astype(float).tolist())
    
    # Add the last matrix
    if current_service and current_matrix:
        matrices[current_service] = np.array(current_matrix)
    
    return matrices

def display_and_calculate(matrices):
    for service, matrix in matrices.items():
        print(f"\nService: {service}")
        print("Matrix:")
        print(matrix)
        
        weights, ci, cr = ahp_weights_and_consistency(matrix)
        
        print("\nResults:")
        print(f"Weights: {weights}")
        print(f"Consistency Index: {ci}")
        print(f"Consistency Ratio: {cr}")
        print(f"{'Consistent' if cr < 0.1 else 'Inconsistent'}")
        print("-" * 50)

# Main execution
if __name__ == "__main__":
    file_path = "simulation_data/pairwise_matrices.csv"  # Replace with your actual file path
    matrices = read_and_process_matrices(file_path)
    matrices
    #display_and_calculate(matrices)

In [278]:
#df.fillna("", inplace = True)
#df.drop(index=df.index[0], axis=0, inplace=True)
#df = df[df.index.notnull()]

df = pd.read_csv(file_path,header = None)
print(df)
df.dropna(inplace = True)

                   0      1      2      3      4
0               VoIP    NaN    NaN    NaN    NaN
1           Criteria      D      J      L      T
2                  D      1      5      3      7
3                  J  0.200      1  0.333      3
4                  L  0.333      3      1      5
5                  T  0.143  0.333  0.200      1
6                NaN    NaN    NaN    NaN    NaN
7    Video Streaming    NaN    NaN    NaN    NaN
8           Criteria      D      J      L      T
9                  D      1      3  0.333  0.200
10                 J  0.333      1  0.200  0.200
11                 L      3      5      1  0.333
12                 T      5      5      3      1
13               NaN    NaN    NaN    NaN    NaN
14      Cloud Gaming    NaN    NaN    NaN    NaN
15          Criteria      D      J      L      T
16                 D      1      3      5      7
17                 J   0.33      1      3      5
18                 L  0.200  0.333      1      3
19                 T

In [279]:
rows_per_matrix = len(df) // 4

# Split the DataFrame into four parts
df1 = df.iloc[:rows_per_matrix].iloc[1:5, 1:5]
df2 = df.iloc[rows_per_matrix:2 * rows_per_matrix].iloc[1:5, 1:5]
df3 = df.iloc[2 * rows_per_matrix:3 * rows_per_matrix].iloc[1:5, 1:5]
df4 = df.iloc[3 * rows_per_matrix:].iloc[1:5, 1:5]

voip = df1.values.astype(np.float64)
video_stream = df2.values.astype(np.float64)
cloud_gaming = df3.values.astype(np.float64)
equal_import = df4.values.astype(np.float64)

In [255]:


weights, ci, cr = ahp_weights_and_consistency(df1)
        
print("\nResults:")
print(f"Weights: {weights}")
print(f"Consistency Index: {ci}")
print(f"Consistency Ratio: {cr}")
print(f"{'Consistent' if cr < 0.1 else 'Inconsistent'}")
print("-" * 50)weights, ci, cr = ahp_weights_and_consistency(df1)
        
print("\nResults:")
print(f"Weights: {weights}")
print(f"Consistency Index: {ci}")
print(f"Consistency Ratio: {cr}")
print(f"{'Consistent' if cr < 0.1 else 'Inconsistent'}")
print("-" * 50)



array([[1.   , 5.   , 3.   , 7.   ],
       [0.2  , 1.   , 0.333, 3.   ],
       [0.333, 3.   , 1.   , 5.   ],
       [0.143, 0.333, 0.2  , 1.   ]])

array([[1.   , 5.   , 3.   , 7.   ],
       [0.2  , 1.   , 0.333, 3.   ],
       [0.333, 3.   , 1.   , 5.   ],
       [0.143, 0.333, 0.2  , 1.   ]])

In [264]:
weights, ci, cr = ahp_weights_and_consistency(voip)
print("\nResults:")
print(f"Weights: {weights}")
print(f"Consistency Index: {ci}")
print(f"Consistency Ratio: {cr}")
print(f"{'Consistent' if cr < 0.1 else 'Inconsistent'}")
print("-" * 50)

0.9

Results:
Weights: [0.56503178 0.1174953  0.26217276 0.05530016]
Consistency Index: 0.038936217353653824
Consistency Ratio: 0.04326246372628203
Consistent
--------------------------------------------------
